# 動画分類(3DCNN, ECO)

In [8]:
!gpustat

Error on querying NVIDIA devices. Use --debug flag for details


In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [50]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# device = "cpu"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [12]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 9-1 動画データに対するディープラーニングとECOの概要
Efficient Convolution Network for Online Video Understanding

### データをディープラーニングで扱う際の注意点
動画を静止画の連なりとして(C,H,W)の三次元のテンソルに対してもう一次元足すだけではうまくいかない．その理由としては時間方向のゆらぎについてカバーできない，つまり同じ動作をしていてもその動作にかかる時間は全く同じであることはないのでそれぞれの動作を別物として処理してしまう．

### 動画データをディープラーニングで扱う方法
動画データの時間方向でのゆらぎについては静止画における空間方向のゆらぎと同様に考えると，静止画でこのようなゆらぎをCNNで吸収したのと同様に動画データについてもCNNで吸収することが考えることができる．

- ```C3D(Convolutional 3D)```：
従来の静止画の(H,W)に対して時間軸を足した(H,W,T)の三次元テンソルに対して各チャネルごとに三次元フィルタを作用させる．**時間についての概念をネットワークによって学習していく．**

- ```Two-Stream ConvNets```：
時間方向の情報を畳み込み層として用意するのではなく，時間の概念を持つ静止画を別で用意する．この時間の概念を表す静止画には**オプティカルフロー(Optical Flow)**と呼ばれる概念を用いる．オプティカルフローとは静止画内で連続するフレーム間で物体がどれだけ移動したかをベクトルで表現したものである．**時間についての概念を先に与えてしまう**


時間についての情報をネットワークがデータから学習するC3Dの方が良い気がするが実際には大量のデータが必要であったりネットワークパラメータも膨大になる欠点がある．この欠点を解決するのがECOである．

ECOの概要は
1. 動画データに前処理を施す．具体的にはフレームごとに動画を分解して大きさの変更や色情報の標準化
2. 動画データから取り出したフレーム数$n$のデータをそれぞれ2D Netモジュールに入力する．これによって$(n, 3, 224, 224)$→ $(n, 96, 28, 28)$に変更される．実際にはmini_batchも考えた$(64,n, 96, 28, 28)$とかになる．
3. これを三次元畳み込み層に入力することによって一次元の特徴量にする
4. さらにこの一次元特徴量を全結合層に入力することによって例えば400次元など想定しているクラスの数の出力にしてあとはソフトマックスなどを用いてクラス分類を行う



## 9-2 2D Netモジュール(Inception-v2)の実装

### ECOの2D Netモジュールの概要
以下の４つのモジュールによって構成される
- BasicConv
- InceptionA
- InceptionB
- InceptionC

### BasicConvモジュールの実装
通常のCNNと同じ．$(3,224,224)\rightarrow (192, 28, 28)$

In [13]:
class BasicConv(nn.Module):
    '''ECOの2D Netモジュールの最初のモジュール'''

    def __init__(self):
        super(BasicConv, self).__init__()

        self.conv1_7x7_s2 = nn.Conv2d(3, 64, kernel_size=(
            7, 7), stride=(2, 2), padding=(3, 3))
        self.conv1_7x7_s2_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv1_relu_7x7 = nn.ReLU(inplace=True)
        self.pool1_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        self.conv2_3x3_reduce = nn.Conv2d(
            64, 64, kernel_size=(1, 1), stride=(1, 1))
        self.conv2_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.conv2_3x3 = nn.Conv2d(64, 192, kernel_size=(
            3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2_3x3_bn = nn.BatchNorm2d(
            192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3 = nn.ReLU(inplace=True)
        self.pool2_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)

    def forward(self, x):
        out = self.conv1_7x7_s2(x)
        out = self.conv1_7x7_s2_bn(out)
        out = self.conv1_relu_7x7(out)
        out = self.pool1_3x3_s2(out)
        out = self.conv2_3x3_reduce(out)
        out = self.conv2_3x3_reduce_bn(out)
        out = self.conv2_relu_3x3_reduce(out)
        out = self.conv2_3x3(out)
        out = self.conv2_3x3_bn(out)
        out = self.conv2_relu_3x3(out)
        out = self.pool2_3x3_s2(out)
        return out


### InceptionA から InceptionCモジュールの実装
**```Inception```：フィルタサイズの大きな畳み込み層を使用するのではなく，フィルタサイズの小さな畳み込み層を並列に使用して学習させるパラメータを減らす** .GoogLeNetで考案された手法．

- InceptionではSelf-Attention GANでも利用した$1\times 1$Convolutions(pointwise convolution)を利用する．各並列の塊の一番最初の層に用いられている．大きすぎるチャネルを小さくするために用いられている．


- 今回はInceptionのversion 2を用いているので並列処理の３つ目の```self.inception3_```には$3\times 3$の畳み込み層を二回作用させている．version1では$5\times 5$を一回作用させていた．


- ４つ目の層ではプーリングにマックスプーリングではなくアベレージプーリングが用いられている

In [14]:
class InceptionA(nn.Module):
    '''InceptionA'''

    def __init__(self):
        super(InceptionA, self).__init__()

        self.inception_3a_1x1 = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_1x1 = nn.ReLU(inplace=True)

        self.inception_3a_3x3_reduce = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_3x3 = nn.Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_3x3_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3 = nn.ReLU(inplace=True)

        self.inception_3a_double_3x3_reduce = nn.Conv2d(
            192, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3a_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_2 = nn.ReLU(inplace=True)

        self.inception_3a_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3a_pool_proj = nn.Conv2d(
            192, 32, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3a_pool_proj_bn = nn.BatchNorm2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_pool_proj = nn.ReLU(inplace=True)

    def forward(self, x):

        out1 = self.inception_3a_1x1(x)
        out1 = self.inception_3a_1x1_bn(out1)
        out1 = self.inception_3a_relu_1x1(out1)

        out2 = self.inception_3a_3x3_reduce(x)
        out2 = self.inception_3a_3x3_reduce_bn(out2)
        out2 = self.inception_3a_relu_3x3_reduce(out2)
        out2 = self.inception_3a_3x3(out2)
        out2 = self.inception_3a_3x3_bn(out2)
        out2 = self.inception_3a_relu_3x3(out2)

        out3 = self.inception_3a_double_3x3_reduce(x)
        out3 = self.inception_3a_double_3x3_reduce_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_reduce(out3)
        out3 = self.inception_3a_double_3x3_1(out3)
        out3 = self.inception_3a_double_3x3_1_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_1(out3)
        out3 = self.inception_3a_double_3x3_2(out3)
        out3 = self.inception_3a_double_3x3_2_bn(out3)
        out3 = self.inception_3a_relu_double_3x3_2(out3)

        out4 = self.inception_3a_pool(x)
        out4 = self.inception_3a_pool_proj(out4)
        out4 = self.inception_3a_pool_proj_bn(out4)
        out4 = self.inception_3a_relu_pool_proj(out4)

        """並列に処理させた後に結合する"""
        outputs = [out1, out2, out3, out4]   

        return torch.cat(outputs, 1)


In [15]:
class InceptionB(nn.Module):
    '''InceptionB'''

    def __init__(self):
        super(InceptionB, self).__init__()
        
        self.inception_3b_1x1 = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_1x1 = nn.ReLU(inplace=True)

        self.inception_3b_3x3_reduce = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_3x3 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_3x3_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3 = nn.ReLU(inplace=True)

        self.inception_3b_double_3x3_reduce = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3b_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_2 = nn.ReLU(inplace=True)

        self.inception_3b_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3b_pool_proj = nn.Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3b_pool_proj_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_pool_proj = nn.ReLU(inplace=True)

    def forward(self, x):
        
        out1 = self.inception_3b_1x1(x)
        out1 = self.inception_3b_1x1_bn(out1)
        out1 = self.inception_3b_relu_1x1(out1)

        out2 = self.inception_3b_3x3_reduce(x)
        out2 = self.inception_3b_3x3_reduce_bn(out2)
        out2 = self.inception_3b_relu_3x3_reduce(out2)
        out2 = self.inception_3b_3x3(out2)
        out2 = self.inception_3b_3x3_bn(out2)
        out2 = self.inception_3b_relu_3x3(out2)

        out3 = self.inception_3b_double_3x3_reduce(x)
        out3 = self.inception_3b_double_3x3_reduce_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_reduce(out3)
        out3 = self.inception_3b_double_3x3_1(out3)
        out3 = self.inception_3b_double_3x3_1_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_1(out3)
        out3 = self.inception_3b_double_3x3_2(out3)
        out3 = self.inception_3b_double_3x3_2_bn(out3)
        out3 = self.inception_3b_relu_double_3x3_2(out3)

        out4 = self.inception_3b_pool(x)
        out4 = self.inception_3b_pool_proj(out4)
        out4 = self.inception_3b_pool_proj_bn(out4)
        out4 = self.inception_3b_relu_pool_proj(out4)

        outputs = [out1, out2, out3, out4]

        return torch.cat(outputs, 1)


InceptionCではA,Bとは異なり並列処理は行わない．

In [16]:
class InceptionC(nn.Module):
    '''InceptionC'''

    def __init__(self):
        super(InceptionC, self).__init__()

        self.inception_3c_double_3x3_reduce = nn.Conv2d(
            320, 64, kernel_size=(1, 1), stride=(1, 1))
        self.inception_3c_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3c_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.inception_3c_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_1 = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.inception_3c_double_3x3_reduce(x)
        out = self.inception_3c_double_3x3_reduce_bn(out)
        out = self.inception_3c_relu_double_3x3_reduce(out)
        out = self.inception_3c_double_3x3_1(out)
        out = self.inception_3c_double_3x3_1_bn(out)
        out = self.inception_3c_relu_double_3x3_1(out)

        return out


以上をまとめて2DNetモジュールクラスを作る

In [17]:
class ECO_2D(nn.Module):
    
    def __init__(self):
        super(ECO_2D,self).__init__()
        
        self.basic_conv = BasicConv()
        self.inception_a = InceptionA()
        self.inception_b = InceptionB()
        self.inception_c = InceptionC()
        
        
    def forward(self, x):
        out = self.basic_conv(x)
        out = self.inception_a(out)
        out = self.inception_b(out)
        out = self.inception_c(out)
        
        return out

In [18]:
# 動作確認
net = ECO_2D()
net.train()

ECO_2D(
  (basic_conv): BasicConv(
    (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_relu_7x7): ReLU(inplace=True)
    (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3_reduce): ReLU(inplace=True)
    (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3): ReLU(inplace=True)
    (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (inception_a): InceptionA(
    (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), 

In [19]:
from tensorboardX import SummaryWriter

writer = SummaryWriter("./tbX/")

batch_size = 1
dummy_img = torch.rand(batch_size, 3, 224, 224)

writer.add_graph(net, (dummy_img, ))
writer.close()

ModuleNotFoundError: No module named 'tensorboardX'

## 9-3 3D Netモジュール(3DCNN)の実装

### ECOの3D Netモジュールの概要
3D Netモジュールでは$(16, 96, 28, 28)\rightarrow (512)$とする.

3D Netモジュールの構成は以下である
1. テンソルの順序を入れ替える$(16, 96, 28, 28)\rightarrow(96, 16, 28, 28)$に変更する．三次元フィルタを用いるので(T,H,W)の順で情報を持っていた方が扱いやすい
2. ResNetの三次元フィルタに複数回入力する
3. 三次元アベレージプーリングを作用させる．ここでは全結合層の代わりにアベレージプーリングを入力テンソルと同じサイズにして作用させることでチャネルごとに三次元テンソルを一つの値に変換する．このように計算量を減らしたり過学習を避けるために全結合層の代わりに用いられるアベレージプーリング層を**Global Average Pooling**と言われる．

### Resnet_3D_3 の実装
テンソルの順番の入れ替えは3D_Netモジュールのforwardの部分にいれるのでここは単純にResnetの実装をすれば良い

In [20]:
class Resnet_3D_3(nn.Module):
    '''Resnet_3D_3'''

    def __init__(self):
        super(Resnet_3D_3, self).__init__()
        
        self.res3a_2 = nn.Conv3d(96, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res3a_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3a_relu = nn.ReLU(inplace=True)

        self.res3b_1 = nn.Conv3d(128, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res3b_1_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_1_relu = nn.ReLU(inplace=True)
        self.res3b_2 = nn.Conv3d(128, 128, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res3b_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_relu = nn.ReLU(inplace=True)

    def forward(self, x):

        residual = self.res3a_2(x)
        out = self.res3a_bn(residual)
        out = self.res3a_relu(out)

        out = self.res3b_1(out)
        out = self.res3b_1_bn(out)
        out = self.res3b_relu(out)
        out = self.res3b_2(out)

        out += residual

        out = self.res3b_bn(out)
        out = self.res3b_relu(out)

        return out


### Resnet_3D_4 の実装

In [21]:
class Resnet_3D_4(nn.Module):
    '''Resnet_3D_4'''

    def __init__(self):
        super(Resnet_3D_4, self).__init__()

        self.res4a_1 = nn.Conv3d(128, 256, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        self.res4a_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_1_relu = nn.ReLU(inplace=True)
        self.res4a_2 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res4a_down = nn.Conv3d(128, 256, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        
        self.res4a_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_relu = nn.ReLU(inplace=True)
        
        self.res4b_1 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res4b_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_1_relu = nn.ReLU(inplace=True)
        self.res4b_2 = nn.Conv3d(256, 256, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res4b_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.res4a_down(x)

        out = self.res4a_1(x)
        out = self.res4a_1_bn(out)
        out = self.res4a_1_relu(out)

        out = self.res4a_2(out)

        out += residual

        residual2 = out

        out = self.res4a_bn(out)
        out = self.res4a_relu(out)

        out = self.res4b_1(out)

        out = self.res4b_1_bn(out)
        out = self.res4b_1_relu(out)

        out = self.res4b_2(out)

        out += residual2

        out = self.res4b_bn(out)
        out = self.res4b_relu(out)

        return out


### Resnet_3D_5 の実装

In [22]:
class Resnet_3D_5(nn.Module):
    '''Resnet_3D_5'''

    def __init__(self):
        super(Resnet_3D_5, self).__init__()
        
        self.res5a_1 = nn.Conv3d(256, 512, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        self.res5a_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_1_relu = nn.ReLU(inplace=True)
        self.res5a_2 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res5a_down = nn.Conv3d(256, 512, kernel_size=(
            3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        
        self.res5a_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_relu = nn.ReLU(inplace=True)
        
        self.res5b_1 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.res5b_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_1_relu = nn.ReLU(inplace=True)
        self.res5b_2 = nn.Conv3d(512, 512, kernel_size=(
            3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        
        self.res5b_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.res5a_down(x)

        out = self.res5a_1(x)
        out = self.res5a_1_bn(out)
        out = self.res5a_1_relu(out)

        out = self.res5a_2(out)

        out += residual  # res5a

        residual2 = out

        out = self.res5a_bn(out)
        out = self.res5a_relu(out)

        out = self.res5b_1(out)

        out = self.res5b_1_bn(out)
        out = self.res5b_1_relu(out)

        out = self.res5b_2(out)

        out += residual2  # res5b

        out = self.res5b_bn(out)
        out = self.res5b_relu(out)

        return out


### ECO_3D モジュールの実装

In [23]:
class ECO_3D(nn.Module):
    def __init__(self):
        super(ECO_3D, self).__init__()

        # 3D_Resnetジュール
        self.res_3d_3 = Resnet_3D_3()
        self.res_3d_4 = Resnet_3D_4()
        self.res_3d_5 = Resnet_3D_5()

        # Global Average Pooling
        self.global_pool = nn.AvgPool3d(
            kernel_size=(4, 7, 7), stride=1, padding=0)

    def forward(self, x):
        '''
        入力xのサイズtorch.Size([batch_num,frames, 96, 28, 28]))
        '''
        out = torch.transpose(x, 1, 2)  # テンソルの順番入れ替え
        out = self.res_3d_3(out)
        out = self.res_3d_4(out)
        out = self.res_3d_5(out)
        out = self.global_pool(out)
        
        # テンソルサイズを変更
        # torch.Size([batch_num, 512, 1, 1, 1])からtorch.Size([batch_num, 512])へ
        out =out.view(out.size()[0], out.size()[1])
        
        return out


In [24]:
# 動作確認
net = ECO_3D()
net.train()

ECO_3D(
  (res_3d_3): Resnet_3D_3(
    (res3a_2): Conv3d(96, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3a_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3a_relu): ReLU(inplace=True)
    (res3b_1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_1_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_1_relu): ReLU(inplace=True)
    (res3b_2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_relu): ReLU(inplace=True)
  )
  (res_3d_4): Resnet_3D_4(
    (res4a_1): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (res4a_1_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res4a_1_relu): ReLU(inplace=True)
    (res4a_2): Conv3d(2

## 9-4 Kinetivs動画データセットをDataLoaderに実装

### Kinetics動画データセットからECO用のDataLoaderを作成
今まで通りに以下の手順でDataLoaderを作成していく
$$
make\_path\_list\rightarrow preprocessing \rightarrow make\_dataset \rightarrow make\_dataloader
$$

ここで今までは一枚ずつ画像を取り込んでいたが、ここでは動画を切り出しているのでひとまとまりとして呼び出すところだけがここまでとは異なる。

In [35]:
def make_datapath_list(root_path):
    
    video_list = list()
    
    # root_pathにあるクラスの種類とパスを取得
    # ここでは [arm wrestling , bungee jumping]
    class_list = os.listdir(root_path)
    
    for class_list_i in class_list:
        
        class_path = os.path.join(root_path, class_list_i)
        
        for file_name in os.listdir(class_path):
            
            # 拡張子を分ける
            name, ext = os.path.splitext(file_name)
            
            # mp4は無視
            if ext == '.mp4':
                continue
                
            video_img_directory_path = os.path.join(class_path, name)
            
            video_list.append(video_img_directory_path)
            
        return video_list

In [42]:
# 動作確認
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)
print(video_list)

['./data/kinetics_videos/arm wrestling/ehLnj7pXnYE_000027_000037', './data/kinetics_videos/arm wrestling/5JzkrOVhPOw_000027_000037', './data/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028_000038', './data/kinetics_videos/arm wrestling/BdMiTo_OtnU_000024_000034']


以下では前処理を行うクラスを実装するが、それぞれの前処理を行うクラスが呼び出される時には一枚の画像として渡されるのではなく画像のリストとして渡されるのでそこだけ注意して実装する必要がある。

また、今回は訓練を行わないのでデータオーギュ面テーションは省略されている。

In [43]:
class VideoTransform():
    """
    動画を画像にした画像ファイルの前処理クラス。学習時と推論時で異なる動作をします。
    動画を画像に分割しているため、分割された画像たちをまとめて前処理する点に注意してください。
    """

    def __init__(self, resize, crop_size, mean, std):
        self.data_transform = {
            'train': torchvision.transforms.Compose([
                # DataAugumentation()  # 今回は省略
                GroupResize(int(resize)),  # 画像をまとめてリサイズ　
                GroupCenterCrop(crop_size),  # 画像をまとめてセンタークロップ
                GroupToTensor(),  # データをPyTorchのテンソルに
                GroupImgNormalize(mean, std),  # データを標準化
                Stack()  # 複数画像をframes次元で結合させる
            ]),
            'val': torchvision.transforms.Compose([
                GroupResize(int(resize)),  # 画像をまとめてリサイズ　
                GroupCenterCrop(crop_size),  # 画像をまとめてセンタークロップ
                GroupToTensor(),  # データをPyTorchのテンソルに
                GroupImgNormalize(mean, std),  # データを標準化
                Stack()  # 複数画像をframes次元で結合させる
            ])
        }

    def __call__(self, img_group, phase):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img_group)

In [45]:
# 前処理で使用するクラスたちの定義


class GroupResize():
    ''' 画像をまとめてリスケールするクラス。
    画像の短い方の辺の長さがresizeに変換される。
    アスペクト比は保たれる。
    '''
    
    # Image.hogeの部分はfilter
    def __init__(self, resize, interpolation=Image.BILINEAR):
        '''リスケールする処理を用意'''
        self.rescaler = torchvision.transforms.Resize(resize, interpolation)

    def __call__(self, img_group):
        '''リスケールをimg_group(リスト)内の各imgに実施'''
        return [self.rescaler(img) for img in img_group]


class GroupCenterCrop():
    ''' 画像をまとめてセンタークロップするクラス。
        （crop_size, crop_size）の画像を切り出す。
    '''

    def __init__(self, crop_size):
        '''センタークロップする処理を用意'''
        self.ccrop = torchvision.transforms.CenterCrop(crop_size)

    def __call__(self, img_group):
        '''センタークロップをimg_group(リスト)内の各imgに実施'''
        return [self.ccrop(img) for img in img_group]


class GroupToTensor():
    ''' 画像をまとめてテンソル化するクラス。
    '''

    def __init__(self):
        '''テンソル化する処理を用意'''
        self.to_tensor = torchvision.transforms.ToTensor()

    def __call__(self, img_group):
        '''テンソル化をimg_group(リスト)内の各imgに実施
        0から1ではなく、0から255で扱うため、255をかけ算する。
        0から255で扱うのは、学習済みデータの形式に合わせるため
        '''

        return [self.to_tensor(img)*255 for img in img_group]


class GroupImgNormalize():
    ''' 画像をまとめて標準化するクラス。
    '''

    def __init__(self, mean, std):
        '''標準化する処理を用意'''
        self.normlize = torchvision.transforms.Normalize(mean, std)

    def __call__(self, img_group):
        '''標準化をimg_group(リスト)内の各imgに実施'''
        return [self.normlize(img) for img in img_group]


class Stack():
    ''' 画像を一つのテンソルにまとめるクラス。
    '''

    def __call__(self, img_group):
        '''img_groupはtorch.Size([3, 224, 224])を要素とするリスト
        '''
        ret = torch.cat([(x.flip(dims=[0])).unsqueeze(dim=0)
                         for x in img_group], dim=0)  # frames次元で結合
        # x.flip(dims=[0])は色チャネルをRGBからBGRへと順番を変えています（元の学習データがBGRであったため）
        # unsqueeze(dim=0)はあらたにframes用の次元を作成しています

        return ret

In [63]:
# Kinetics-400のラベル名をIDに変換する辞書と、逆にIDをラベル名に変換する辞書を用意
import csv

def get_label_id_dictionary(label_dicitionary_path='./video_download/kinetics_400_label_dicitionary.csv'):
    label_id_dict = {}
    id_label_dict = {}

    with open(label_dicitionary_path, encoding="utf-8_sig") as f:

        # 読み込む
        reader = csv.DictReader(f, delimiter=",", quotechar='"')
#         print(reader.__next__())
#         itr = iter(reader)
#         print(itr.__next__()["class_label"])

        # 1行ずつ読み込み、辞書型変数に追加します
        for row in reader:
            label_id_dict.setdefault(
                row["class_label"], int(row["label_id"])-1)
            id_label_dict.setdefault(
                int(row["label_id"])-1, row["class_label"])

    return label_id_dict,  id_label_dict


# 確認
label_dicitionary_path = './video_download/kinetics_400_label_dicitionary.csv'
label_id_dict, id_label_dict = get_label_id_dictionary(label_dicitionary_path)
label_id_dict

{'abseiling': 0,
 'air drumming': 1,
 'answering questions': 2,
 'applauding': 3,
 'applying cream': 4,
 'archery': 5,
 'arm wrestling': 6,
 'arranging flowers': 7,
 'assembling computer': 8,
 'auctioning': 9,
 'baby waking up': 10,
 'baking cookies': 11,
 'balloon blowing': 12,
 'bandaging': 13,
 'barbequing': 14,
 'bartending': 15,
 'beatboxing': 16,
 'bee keeping': 17,
 'belly dancing': 18,
 'bench pressing': 19,
 'bending back': 20,
 'bending metal': 21,
 'biking through snow': 22,
 'blasting sand': 23,
 'blowing glass': 24,
 'blowing leaves': 25,
 'blowing nose': 26,
 'blowing out candles': 27,
 'bobsledding': 28,
 'bookbinding': 29,
 'bouncing on trampoline': 30,
 'bowling': 31,
 'braiding hair': 32,
 'breading or breadcrumbing': 33,
 'breakdancing': 34,
 'brush painting': 35,
 'brushing hair': 36,
 'brushing teeth': 37,
 'building cabinet': 38,
 'building shed': 39,
 'bungee jumping': 40,
 'busking': 41,
 'canoeing or kayaking': 42,
 'capoeira': 43,
 'carrying baby': 44,
 'cartw

いつも```torch.utils.data.Dataset```を継承する

```__getitem__(self,index)```の内部で扱いやすいように別で```pull_item()```を呼び出してこれを定義している

In [48]:
class VideoDataset(torch.utils.data.Dataset):
    """
    動画のDataset
    """

    def __init__(self, video_list, label_id_dict, num_segments, phase, transform, img_tmpl='image_{:05d}.jpg'):
        self.video_list = video_list  # 動画画像のフォルダへのパスリスト
        self.label_id_dict = label_id_dict  # ラベル名をidに変換する辞書型変数
        self.num_segments = num_segments  # 動画を何分割して使用するのかを決める
        self.phase = phase  # train or val
        self.transform = transform  # 前処理
        self.img_tmpl = img_tmpl  # 読み込みたい画像のファイル名のテンプレート

    def __len__(self):
        '''動画の数を返す'''
        return len(self.video_list)

    def __getitem__(self, index):
        '''
        前処理をした画像たちのデータとラベル、ラベルIDを取得
        '''
        imgs_transformed, label, label_id, dir_path = self.pull_item(index)
        return imgs_transformed, label, label_id, dir_path

    def pull_item(self, index):
        '''前処理をした画像たちのデータとラベル、ラベルIDを取得'''

        # 1. 画像たちをリストに読み込む
        dir_path = self.video_list[index]  # 画像が格納されたフォルダ
        indices = self._get_indices(dir_path)  # 読み込む画像idxを求める
        img_group = self._load_imgs(
            dir_path, self.img_tmpl, indices)  # リストに読み込む

        # 2. ラベルの取得し、idに変換する
        label = (dir_path.split('/')[3].split('/')[0])
        label_id = self.label_id_dict[label] # idを取得

        # 3. 前処理を実施
        imgs_transformed = self.transform(img_group, phase=self.phase)

        return imgs_transformed, label, label_id, dir_path

    def _load_imgs(self, dir_path, img_tmpl, indices):
        '''画像をまとめて読み込み、リスト化する関数'''
        img_group = []  # 画像を格納するリスト

        for idx in indices:
            # 画像のパスを取得
            file_path = os.path.join(dir_path, img_tmpl.format(idx))

            # 画像を読み込む
            img = Image.open(file_path).convert('RGB')

            # リストに追加
            img_group.append(img)
        return img_group

    def _get_indices(self, dir_path):
        """
        動画全体をself.num_segmentに分割した際に取得する動画のidxのリストを取得する
        """
        # 動画のフレーム数を求める
        file_list = os.listdir(path=dir_path)
        num_frames = len(file_list)

        # 動画の取得間隔幅を求める
        tick = (num_frames) / float(self.num_segments)
        # 250 / 16 = 15.625
        # 動画の取得間隔幅で取り出す際のidxをリストで求める
        indices = np.array([int(tick / 2.0 + tick * x)
                            for x in range(self.num_segments)])+1
        # 250frameで16frame抽出の場合
        # indices = [  8  24  40  55  71  86 102 118 133 149 165 180 196 211 227 243]

        return indices



In [64]:
# 動作確認

# vieo_listの作成
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)

# 前処理の設定
resize, crop_size = 224, 224
mean, std = [104, 117, 123], [1, 1, 1]
video_transform = VideoTransform(resize, crop_size, mean, std)

# Datasetの作成
# num_segments は 動画を何分割して使用するのかを決める
val_dataset = VideoDataset(video_list, label_id_dict, num_segments=16,
                           phase="val", transform=video_transform, img_tmpl='image_{:05d}.jpg')

# データの取り出し例
# 出力は、imgs_transformed, label, label_id, dir_path
index = 0
print(val_dataset.__getitem__(index)[0].shape)  # 画像たちのテンソル
print(val_dataset.__getitem__(index)[1])  # ラベル名
print(val_dataset.__getitem__(index)[2])  # ラベルID
print(val_dataset.__getitem__(index)[3])  # 動画へのパス

torch.Size([16, 3, 224, 224])
arm wrestling
6
./data/kinetics_videos/arm wrestling/ehLnj7pXnYE_000027_000037


In [70]:
# DataLoaderにします
batch_size = 8
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 動作確認
batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す
print(imgs_transformeds.shape)

torch.Size([4, 16, 3, 224, 224])


## 9-5 ECOモデルの実装と動画分類の推論実施

### ECOモデルの実装
上で作ったECO_2DとECO_3Dをくっつける

ここで注意が必要なのは、```nn.Conv2D```は四次元テンソルまでしか入力することができないので今回はframe数も入れた五次元テンソルとなってしまうのでこの$(batch\_size\times frames)$を掛け合わせて一次元にして処理を行って後からこれらを分解する。

In [83]:
class ECO_Lite(nn.Module):
    def __init__(self):
        super(ECO_Lite, self).__init__()
        
        self.eco_2d = ECO_2D()
        self.eco_3d = ECO_3D()
        self.fc_final = nn.Linear(in_features=512, out_features=400, bias=True)
        
    def forward(self, x):
        
        # 後で戻すように保存しておく
        _, ns, c, h, w = x.shape
        
        # 4次元テンソルまでしか処理できないので無理やり変形して後で戻す
        out = x.view(-1, c, h, w)
        
        out = self.eco_2d(out)
        
        _, nc, nh, nw = out.shape
        
        out = out.view(-1, ns, nc, nh, nw)
        
        out = self.eco_3d(out)
        out = self.fc_final(out)
        
        return out

In [84]:
net = ECO_Lite()
print(net)

ECO_Lite(
  (eco_2d): ECO_2D(
    (basic_conv): BasicConv(
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1_relu_7x7): ReLU(inplace=True)
      (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3_reduce): ReLU(inplace=True)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2_relu_3x3): ReLU(inplace=True)
      (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (inception_a): InceptionA(
      (inc

### 学習済みモデルをロード
公開されている学習済みモデルに付けられている名前が、ここで実装したモデルと異なる部分があるのでそのままではモデルをロードすることができない。モデルで使用されてるパラメータの種類や順番は同じなので工夫してこれらを読み込む。

In [91]:
pretrained_model = torch.load(net_model_ECO, map_location='cpu')
pretrained_model

{'epoch': 53,
 'best_prec1': 61.77472489922216,
 'state_dict': OrderedDict([('module.base_model.conv1_7x7_s2.weight',
               tensor([[[[ 2.5400e-02, -1.4457e-02, -4.5913e-02,  ..., -1.1885e-02,
                           8.3576e-03,  1.3171e-02],
                         [-1.9247e-02, -6.9981e-02, -9.2102e-02,  ..., -3.6535e-03,
                           1.5782e-02,  1.4442e-02],
                         [-6.0236e-02, -9.6394e-02, -6.8235e-02,  ...,  7.0038e-02,
                           6.0445e-02,  2.9128e-02],
                         ...,
                         [-8.8571e-03,  2.6555e-02,  9.0340e-02,  ...,  1.1250e-01,
                           2.9375e-02, -3.7008e-02],
                         [ 6.6437e-03,  3.7149e-02,  6.8393e-02,  ...,  2.4852e-02,
                          -3.4692e-02, -6.5817e-02],
                         [ 6.7576e-04,  1.9097e-02,  2.4318e-02,  ..., -2.1916e-02,
                          -4.9671e-02, -4.1157e-02]],
               
             

In [85]:
# 学習済みモデルをロードする関数の定義


def load_pretrained_ECO(model_dict, pretrained_model_dict):
    '''ECOの学習済みモデルをロードする関数
    今回構築したECOは学習済みモデルとレイヤーの順番は同じだが名前が異なる
    '''

    # 現在のネットワークモデルのパラメータ名
    param_names = []  # パラメータの名前を格納していく
    for name, param in model_dict.items():
        param_names.append(name)

    # 現在のネットワークの情報をコピーして新たなstate_dictを作成
    new_state_dict = model_dict.copy()

    # 新たなstate_dictに学習済みの値を代入
    print("学習済みのパラメータをロードします")
    for index, (key_name, value) in enumerate(pretrained_model_dict.items()):
        name = param_names[index]  # 現在のネットワークでのパラメータ名を取得
        new_state_dict[name] = value  # 値を入れる

        # 何から何にロードされたのかを表示
        print(str(key_name)+"→"+str(name))

    return new_state_dict


# 学習済みモデルをロード
net_model_ECO = "./weights/ECO_Lite_rgb_model_Kinetics.pth.tar"
pretrained_model = torch.load(net_model_ECO, map_location='cpu')
pretrained_model_dict = pretrained_model['state_dict']
# （注釈）
# pthがtarで圧縮されているのは、state_dict以外の情報も一緒に保存されているため。
# そのため読み込むときは辞書型変数になっているので['state_dict']で指定する。

# 現在のモデルの変数名などを取得
model_dict = net.state_dict()

# 学習済みモデルのstate_dictを取得
new_state_dict = load_pretrained_ECO(model_dict, pretrained_model_dict)

# 学習済みモデルのパラメータを代入
net.eval()  # ECOネットワークを推論モードに
net.load_state_dict(new_state_dict)

学習済みのパラメータをロードします
module.base_model.conv1_7x7_s2.weight→eco_2d.basic_conv.conv1_7x7_s2.weight
module.base_model.conv1_7x7_s2.bias→eco_2d.basic_conv.conv1_7x7_s2.bias
module.base_model.conv1_7x7_s2_bn.weight→eco_2d.basic_conv.conv1_7x7_s2_bn.weight
module.base_model.conv1_7x7_s2_bn.bias→eco_2d.basic_conv.conv1_7x7_s2_bn.bias
module.base_model.conv1_7x7_s2_bn.running_mean→eco_2d.basic_conv.conv1_7x7_s2_bn.running_mean
module.base_model.conv1_7x7_s2_bn.running_var→eco_2d.basic_conv.conv1_7x7_s2_bn.running_var
module.base_model.conv1_7x7_s2_bn.num_batches_tracked→eco_2d.basic_conv.conv1_7x7_s2_bn.num_batches_tracked
module.base_model.conv2_3x3_reduce.weight→eco_2d.basic_conv.conv2_3x3_reduce.weight
module.base_model.conv2_3x3_reduce.bias→eco_2d.basic_conv.conv2_3x3_reduce.bias
module.base_model.conv2_3x3_reduce_bn.weight→eco_2d.basic_conv.conv2_3x3_reduce_bn.weight
module.base_model.conv2_3x3_reduce_bn.bias→eco_2d.basic_conv.conv2_3x3_reduce_bn.bias
module.base_model.conv2_3x3_reduce_bn.ru

<All keys matched successfully>

In [86]:
# 推論します
net.eval()  # ECOネットワークを推論モードに

batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(
    batch_iterator)  # 1番目の要素を取り出す

with torch.set_grad_enabled(False):
    outputs = net(imgs_transformeds)  # ECOで推論

print(outputs.shape)  # 出力のサイズ

torch.Size([4, 400])


In [87]:
# 予測結果の上位5つを表示します
def show_eco_inference_result(dir_path, outputs_input, id_label_dict, idx=0):
    '''ミニバッチの各データに対して、推論結果の上位を出力する関数を定義'''
    print("ファイル：", dir_path[idx])  # ファイル名

    outputs = outputs_input.clone()  # コピーを作成

    for i in range(5):
        '''1位から5位までを表示'''
        output = outputs[idx]
        _, pred = torch.max(output, dim=0)  # 確率最大値のラベルを予測
        class_idx = int(pred.numpy())  # クラスIDを出力
        print("予測第{}位：{}".format(i+1, id_label_dict[class_idx]))
        outputs[idx][class_idx] = -1000  # 最大値だったものを消す（小さくする）


# 予測を実施
idx = 0
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)

ファイル： ./data/kinetics_videos/arm wrestling/ehLnj7pXnYE_000027_000037
予測第1位：arm wrestling
予測第2位：tickling
予測第3位：hugging
予測第4位：headbutting
予測第5位：punching person (boxing)


In [89]:
# 予測を実施
idx = 3
show_eco_inference_result(dir_path, outputs, id_label_dict, idx)

ファイル： ./data/kinetics_videos/arm wrestling/BdMiTo_OtnU_000024_000034
予測第1位：arm wrestling
予測第2位：waxing legs
予測第3位：getting a tattoo
予測第4位：drinking beer
予測第5位：stretching leg
